In [ ]:
#!/usr/bin/env python3
"""
Battery State Simulator
Mensimulasikan battery drain berdasarkan robot activity
"""

import rclpy
from rclpy.node import Node
from sensor_msgs.msg import BatteryState
from geometry_msgs.msg import Twist
import math


class BatterySimulator(Node):
    """Simulate battery discharge for the robot"""

    def __init__(self):
        super().__init__('battery_simulator')

        # Publisher
        self.battery_pub = self.create_publisher(BatteryState, '/battery_state', 10)

        # Subscriber
        self.create_subscription(Twist, '/cmd_vel', self.cmd_vel_callback, 10)

        # Battery state
        self.battery_percentage = 1.0  # 100%
        self.battery_voltage = 12.0  # Volts
        self.is_moving = False
        self.is_charging = False
        self.charge_position = [0.0, 0.0]

        # Parameters
        self.declare_parameter('discharge_rate_idle', 0.0001)  # %/s when idle
        self.declare_parameter('discharge_rate_moving', 0.0005)  # %/s when moving
        self.declare_parameter('charge_rate', 0.002)  # %/s when charging
        self.declare_parameter('charge_distance_threshold', 0.5)  # meters
        self.declare_parameter('publish_rate', 1.0)  # Hz

        # Get parameters
        self.discharge_idle = self.get_parameter('discharge_rate_idle').value
        self.discharge_moving = self.get_parameter('discharge_rate_moving').value
        self.charge_rate = self.get_parameter('charge_rate').value
        publish_rate = self.get_parameter('publish_rate').value

        # Timer for publishing battery state
        self.timer = self.create_timer(1.0 / publish_rate, self.publish_battery_state)

        # Timer for battery simulation
        self.sim_timer = self.create_timer(0.1, self.update_battery)

        self.get_logger().info('Battery Simulator started')
        self.get_logger().info(f'Initial battery: {self.battery_percentage * 100:.1f}%')

    def cmd_vel_callback(self, msg):
        """Check if robot is moving"""
        linear_vel = abs(msg.linear.x) + abs(msg.linear.y) + abs(msg.linear.z)
        angular_vel = abs(msg.angular.z)

        self.is_moving = (linear_vel > 0.01 or angular_vel > 0.01)

    def update_battery(self):
        """Update battery level based on activity"""
        dt = 0.1  # Timer period

        # Check if at charging station (simplified - assume at origin)
        # In real implementation, would check robot position from /odom
        # For now, assume charging when battery < 30% and not moving for a while
        if self.battery_percentage < 0.3 and not self.is_moving:
            self.is_charging = True
        elif self.battery_percentage > 0.95:
            self.is_charging = False

        if self.is_charging:
            # Charging
            self.battery_percentage = min(1.0, self.battery_percentage + self.charge_rate * dt)
            self.battery_voltage = 12.0 + (self.battery_percentage * 2.0)  # 12-14V range
        else:
            # Discharging
            if self.is_moving:
                self.battery_percentage = max(0.0, self.battery_percentage - self.discharge_moving * dt)
            else:
                self.battery_percentage = max(0.0, self.battery_percentage - self.discharge_idle * dt)

            self.battery_voltage = 12.0 + (self.battery_percentage * 2.0)

    def publish_battery_state(self):
        """Publish current battery state"""
        msg = BatteryState()

        # Header
        msg.header.stamp = self.get_clock().now().to_msg()
        msg.header.frame_id = 'base_link'

        # Battery info
        msg.voltage = self.battery_voltage
        msg.temperature = float('nan')
        msg.current = float('nan')
        msg.charge = float('nan')
        msg.capacity = float('nan')
        msg.design_capacity = float('nan')
        msg.percentage = self.battery_percentage

        # Power supply status
        if self.is_charging:
            msg.power_supply_status = BatteryState.POWER_SUPPLY_STATUS_CHARGING
        elif self.battery_percentage < 0.2:
            msg.power_supply_status = BatteryState.POWER_SUPPLY_STATUS_DISCHARGING
        else:
            msg.power_supply_status = BatteryState.POWER_SUPPLY_STATUS_NOT_CHARGING

        msg.power_supply_health = BatteryState.POWER_SUPPLY_HEALTH_GOOD
        msg.power_supply_technology = BatteryState.POWER_SUPPLY_TECHNOLOGY_LION
        msg.present = True

        self.battery_pub.publish(msg)

        # Log occasionally
        if self.battery_percentage < 0.2:
            self.get_logger().warn(
                f'Battery LOW: {self.battery_percentage * 100:.1f}%',
                throttle_duration_sec=5.0
            )
        elif self.battery_percentage < 0.5:
            self.get_logger().info(
                f'Battery: {self.battery_percentage * 100:.1f}%',
                throttle_duration_sec=10.0
            )


def main(args=None):
    rclpy.init(args=args)
    node = BatterySimulator()

    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()